# Dash Testing

In [ ]:
from dash import Dash, html, dcc, Input, Output
import altair as alt
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np
import matplotlib as plt
from matplotlib.pyplot import figure

import sheworewhat as sww

alt.data_transformers.disable_max_rows()

closet = sww.closet_df()
worn_df = sww.complete_df(closet)
top_10, heat_df = sww.top_10_df()

def plot_mostworn():
    most_worn = worn_df.nlargest(15, columns="Count")
    closet_comp = alt.Chart(most_worn, title="2023 Most Worn Pieces"
                       ).mark_bar(color="#6ce4d8"
                         ).encode(alt.X("Name",
                                        title="",
                                        axis=alt.Axis(labelAngle=-45), 
                                        sort="-y"),
                                  alt.Y("Count", 
                                        title="Times Worn",
                                        axis=alt.Axis(tickMinStep=1)),
                                  alt.Tooltip("Count")
                                 ).configure_axis(grid=False, domain=False)
                
    return closet_comp.to_html()

def plot_facet():
    
    categories = ["Top", "Bottom", "Full Body", "Outerwear", "Accessory", "Shoes"]

    cat_plots = []

    for i in categories:
        category_worn = worn_df.loc[worn_df["Category"] == i].nlargest(5, columns="Count")

        category_plot = alt.Chart(category_worn, title=f"2023 Most Worn {i}"
                           ).mark_bar(color="#827191",
                           ).encode(alt.X("Name", title="", axis=alt.Axis(labelAngle=-45), sort="-y"),
                                    alt.Y("Count", 
                                    title="# of Times Worn",
                                    axis=alt.Axis(tickMinStep=1)),
                                    alt.Tooltip(["Name", "Count"])
                            ).properties(
                                height=200,
                                width=150
                                )
        cat_plots.append(category_plot)

    # configure altair charts
    row1 = alt.hconcat(cat_plots[0], cat_plots[1], cat_plots[2])
    row2 = alt.hconcat(cat_plots[3], cat_plots[4], cat_plots[5])

    category_plot = alt.vconcat(row1, row2).configure_axis(grid=False, domain=False)
    return category_plot.to_html()

app = Dash(__name__)
server = app.server

app.title = "She Wore What 2023"

app.layout = dbc.Container(
    [
        html.Br(),
        dbc.Row(
            dbc.Col(
                dbc.Card(
                    dbc.CardBody(
                        html.B("She Wore What 2023"),
                    ), 
                ),
                style={"font-weight": "bold", 
                       "color": "#000000", 
                       "font-family" :"Garamond",
                      "font-size" : "120%"},
                className='text-center mt-4 mb-5'
            )
        ),
        dbc.Row(
            dbc.Card(dbc.CardBody([
                html.Div([
                    html.Iframe(
                        id='mostworm',
                        style={'border-width': '0', 'width': '100%', 'height': '600px'},
                        srcDoc=plot_mostworn()
                        )
                ]),
            ])
        )),
        dbc.Col(
            dbc.Row(
                dbc.Card(
                    dbc.CardBody([
                        html.B("Title2")
                    ]),
                )
            ),
#             dbc.Row(
#                 dbc.Card(
#                     dbc.CardBody([
#                         html.Div([
#                             html.Iframe(
#                                 id='mostworm',
#                                 style={'border-width': '0', 'width': '100%', 'height': '600px'},
#                                 srcDoc=plot_facet()
#                             )   
#                          ])
#                     ])
#                 )
#             )
        )
    ])

if __name__ == '__main__':
    app.run_server(
        port = 8073,
        debug=False)

Dash is running on http://127.0.0.1:8073/

Dash is running on http://127.0.0.1:8073/

Dash is running on http://127.0.0.1:8073/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8073
Press CTRL+C to quit
127.0.0.1 - - [30/Dec/2022 22:01:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2022 22:01:54] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2022 22:01:54] "GET /_dash-dependencies HTTP/1.1" 200 -
